In [1]:
import gzip
import pandas
import xml.etree.ElementTree as ET

In [2]:
boern_xml = 'Boern.xml.gz'
with gzip.open(boern_xml) as fin:
    tree = ET.parse(fin)
    root = tree.getroot()
    child_data = []
    for child in root:
        #print(child.tag, child.attrib)
        child_attrib_list = [0] * 7
        for i, child_attrib in enumerate(child):
            #print(child_attrib.tag, child_attrib.text)
            child_attrib_list[i] = child_attrib.text
        child_data.append(child_attrib_list)

child_df = pandas.DataFrame(child_data,
                            columns=['Child', 'Year', 'Gender', 'District',
                                     'Weight', 'Height', 'Head Circumference'])

In [3]:
print('Read {:d} entries for {:d} children.'.format(len(child_df), len(child_df['Child'].unique())))

Read 116242 entries for 26125 children.
